$\lambda$-calculus is known as the world's tiniest universal programming language that adopts the functional programming paradigm.

When I first learned its concepts, I was frustrated a lot about its confusing formation and variable substitutions. And I felt it was hard to understand _intuitively_ the fundamental functions defined in the language.

After some effort, I realized this language shared the same spirit with many other languages (in fact, this one is the foundation) that support functional programming paradigm, such as Python. It took only a subset concepts from the more complete languages, yet ensured the language was still Turing complete.

Let's quickly review the basic concepts of $\lambda$-calculus.

## Basic concept of $\lambda$-calculus

In a single sentence, $\lambda$-calculus is purely about operations (or `functions`). Any other concepts are built upon functions, such as `numbers`, `conditional logics`, `bools`. I somehow tasted a flavor of minimalism.

The core idea of $\lambda$-calculus is that by defining several fundamental operations (a.k.a functions) properly, we can pretty much build the whole world upon the fundamental functions. This is exactly the spirit of functional programming, to the extreme extent.

* __variable__ or __name__: a representation of `function`, or `application`. 

* __function__: the implementation of `function` using `variable`s.  

* __application__: the calling of `function` on some other `variable`s.

Please review the [introductory material](http://www.inf.fu-berlin.de/lehre/WS03/alpi/lambda.pdf) to understand the following examples.

### Some classic examples

1. numbers as functions

    $$0 \equiv \lambda sz.z$$  
    $$1 \equiv \lambda sz.s(z)$$
    
2. bools as functions

    $$T \equiv \lambda xy.x$$  
    $$F \equiv \lambda xy.y$$
    
3. conditional logic as functions

    $$ifelse \equiv \lambda cxy.cxy$$
    
Looks like magical right? Not until you literally read out their operations and turn them into Python.

## Python implementation of $\lambda$-calculus

We have to now change our mindsets to discard any unnecessary concepts in python, such as `operators`, `strings`, `bool`, `numbers`, `if`, `while` and etc. The only keyword we can use is basically `lambda`. And this is exactly where the name of `lambda` function comes from.

### Arithmetic

Let's think about Arithmetic, or more basically numbers. How can we define a number as function? 

Numbers are just symbols, therefore it certainly can be functions. 

First thing we have to realize is that numbers are not isolated, they are connected with other numbers by operations in order to be meaningful. By applying some operation to a number, one number can become another.

Now let's turn this property into the concepts of $\lambda$-calculus.

$number \equiv function$, number is a function  
$function(number) \equiv number \equiv function$, some function applying to a number becomes another number, or another function.

In other words, the function defines a number should be a __higher-order function__.

If we define the rule that a number can be transformed to its successor by a function `s`. Then we can have an iterative function for number $n_i$: $n_i = s(n_{i-1})$, expanding it becomes $n_i = s(s(..(s(n_0))))$, a nth-order function.

#### numbers

In [148]:
# a function representing the fundamental operation: number zero.
# The real meaning of this function is it always return the second parameter.
zero = lambda s: lambda zero: zero

# The real meaning of function one is it applies function s to the second parameter __once__.
one = lambda s: lambda zero: s(zero)

# The real meaning of function two is it applies function s to the second parameter __twice__.
two = lambda s: lambda zero: s(s(zero))

three = lambda s: lambda zero: s(s(s(zero)))

Now we have these basic rules, let's see what the real meanings of functions `zero`, `one`, `two` are. We will use a function `plus_one` for test purpose, meaning it will add 1 to the input number.

In [16]:
plus_one = lambda x: x + 1

Applying function `zero` to any other functions would result in an identity function $\lambda x.x$

In [17]:
identity = zero(plus_one)
identity(0), identity(1)

(0, 1)

Applying function `one` to another function `x` would result in a function that will apply function `x` once to the input.

In [18]:
plus_one_once = one(plus_one)
plus_one_once(0), plus_one_once(1)

(1, 2)

Applying function `two` to another function x would result in a function that will apply function x twice to the input.

In [19]:
plus_one_twice = two(plus_one)
plus_one_twice(0), plus_one_twice(1)

(2, 3)

#### successor

`plus_one` in this case also means the `successor` of the current number. The definition does not meet the requirement of $\lambda$-calculus, because it uses `numbers`. Can we define a $\lambda$-calculus `plus_one`(a.k.a `successor`) function, so that we can get function `two` from `one` or function `three` from `two`? 

Let us analyze the role of `successor` in terms of the operation it represents (not in an arithmetic sense). Literally, it means transforming a number function `n` so that `n` applies function `s` one more time than `n` to the second parameter `m`. By this definition, we can define a function as following,

In [20]:
successor = lambda n: (lambda s: lambda m: s(n(s)(m)))

`n(s)(m)` means applying `s` to `m` for `n` times.  
`lambda m: s(n(s)(m)))` means applying `s` one more time to `m`.

In $\lambda$-calculus form,

$$ S = \lambda nsm.s(n(sm))$$

In [21]:
new_two = successor(one)

In [22]:
new_plus_one_twice = new_two(plus_one)
original_plus_one_twice = two(plus_one)
new_plus_one_twice(100) == original_plus_one_twice(100), new_plus_one_twice(100)

(True, 102)

It can be chained.

In [91]:
new_three = successor(successor(successor(zero)))
new_plus_one_triple_times = new_three(plus_one)
new_plus_one_triple_times(100)

103

In [92]:
new_three = successor(two)
new_plus_one_triple_times = new_three(plus_one)
new_plus_one_triple_times(100)

103

Based on `numbers` and `successor` functions, more complex arithmetic functions such as `add`, `mimus`, `preccessor`, `multiple`, `divide` may be defined.

#### predecessor

The strategy to design a $\lambda$-calculus function of predecessor for numbers is different from successor, given that we can only use `successor` and `number` functions.

Predecessor of `number` function $n = \lambda sz.s(...s(z))$ may be literally described as,

> Transform function `n` so that it applies one less time function `s` to variable `z`.

Because we only have access to function `zero`, `n` and `successor` function, this definition can also be described as,

> Generate a sequence of number functions from `zero` to `n`, and select the second to the last variable `n-1`.

Generating a sequence of number functions may be also described in iterative form with pairs of adjacent number functions $(n, n-1)$. The new iterative function $\theta$ transforms $(n, n-1)$ to $(n+1, n)$. Iteratively, starting from a pair of $(1, 0) = (successor(0), 0)$. Then the predecessor function may be described as,

> Applying function $\theta$ for `n` times to the initial function $(1, 0)$ to get pair $(n, n-1)$, and then select the second element in the pair.

##### Pair function

Literally, pair function means applying function `z` to two variables `a` and `b`.

$$ p = \lambda z.zab $$

In Python, it becomes,

In [134]:
pair = lambda a: lambda b: lambda z: z(a)(b)

In [124]:
# this means applying successor to one (becomes two) and then apply two to plus_one

one_and_plus_one_pair = pair(one)(plus_one)
zero_and_plus_one_pair = pair(zero)(plus_one)

plus_one_twice = one_and_plus_one_pair(successor)
plus_one_once = zero_and_plus_one_pair(successor)
plus_one_twice(2), plus_one_once(2)

(4, 3)

##### Selector function

Means that the function can select the first or second element in a pair.

$$ T = \lambda xy.x $$

$$ F = \lambda xy.y $$

In Python, they become,

In [125]:
first = lambda x: lambda y: x
second = lambda x: lambda y: y

In [126]:
plus_one_once = first(one)(two)(plus_one)

# or
plus_one_twice = second(one)(two)(plus_one)

plus_one_once(1), plus_one_twice(1)

(2, 3)

Combining selector and pair function, we can have functions to extract the first or second element in a pair.

In [127]:
one_two_pair = pair(one)(two)
new_one = one_two_pair(first)
new_two = one_two_pair(second)
plus_one_once = new_one(plus_one)
plus_one_twice = new_two(plus_one)

plus_one_once(1), plus_one_twice(1)

(2, 3)

##### $\theta$ function

$\theta$ function transforms a pair to a new pair where the first element is the successor of the first element of the original pair, and second pair is the first element of the original pair.

$$ \theta = \lambda pz.z(S(pT))(pT) $$

The result function would also be a pair function.

In Python,

In [140]:
theta = lambda p: lambda z: pair(successor(p(first)))(p(first))(z)

In [143]:
successor_of_first_and_original_first_pair = theta(pair(zero)(zero))
new_one = successor_of_first_and_original_first_pair(first)
new_zero = successor_of_first_and_original_first_pair(second)
new_one(plus_one)(2), new_zero(plus_one)(2)

(3, 2)

##### Predecessor

Now we have all building blocks `theta`, `pair`, `first` and `second` functions. We may define the predecessor function as following,

$$ P = \lambda n.n\theta(p00)F $$

Or in Python as,

In [144]:
predecessor = lambda n: n(theta)(pair(zero)(zero))(second)

In [149]:
new_zero = predecessor(zero)
new_zero_from_one = predecessor(one)
new_one = predecessor(two)
new_two = predecessor(three)

new_zero(plus_one)(1), new_zero_from_one(plus_one)(1), new_one(plus_one)(1), new_two(plus_one)(1)

(1, 1, 2, 3)